In [138]:
import numpy as np
import pandas as pd
import argparse
from sklearn.tree import DecisionTreeClassifier, plot_tree, _tree
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import pydotplus
from matplotlib import pyplot as plt
from labels import mac_to_label
import json
import math

In [139]:
Set1 = pd.read_csv('data/combined/data.csv').values.tolist()
X = [i[0:3] for i in Set1]
Y =[i[3] for i in Set1]

In [142]:
predict_Yt = []
index=0

with open('compressed_tree.json', 'r') as file:
    data = json.load(file)
    classes = data["classes"]
    for x in X:
        counter = 0
        class_set = []
        paths_set = []
        features = ["protocol", "src", "dst"]
        for feature in features:
            if feature in data["layers"]:
                for node in data['layers'][feature]:
                    if node['min'] is None:
                        if x[counter] <= node['max']:
                            class_set.append(node['classes'])
                            paths_set.append(node["paths"])
                            break #is this an issue?
                        else:
                            continue
                    elif node['max'] is None:
                        if node['min'] < x[counter]:
                            class_set.append(node['classes'])
                            paths_set.append(node["paths"])
                            break #is this an issue?
                        else:
                            continue
                    elif node['min'] < x[counter] and x[counter] <= node['max']:
                        class_set.append(node['classes'])
                        paths_set.append(node["paths"])
                        break #is this an issue?

            counter += 1
        result = set(class_set[0])
        paths = set(paths_set[0])
        for s in class_set[1:]:
            result.intersection_update(s)
        for s in paths_set[1:]:
            paths.intersection_update(s)

        #predict_Yt.append(list(result))
        #print(result)
        if len(paths) != 1:
            print(paths)
            print(x)
            print(result)
        assert len(paths) == 1
        path = list(paths)[0]
        pred = data["path_to_class"][str(path)]
        pred_class = classes[pred]
        predict_Yt.append(pred_class)
        
        index += 1

{'0': 20, '1': 20, '2': 9, '3': 20, '4': 0, '5': 13, '6': 20, '7': 0, '8': 12, '9': 4, '10': 20, '11': 4, '12': 1, '13': 16, '14': 20, '15': 2, '16': 20, '17': 0, '18': 20, '19': 20, '20': 20, '21': 20, '22': 20, '23': 1, '24': 2, '25': 20, '26': 13, '27': 11, '28': 20, '29': 20}


In [143]:
correct = 0
for i in range(len(Y)):
    prediction = predict_Yt[i]
    if prediction != None and Y[i] == prediction:
        correct += 1

print(correct / len(Y))

0.8410252791654538
